In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


# **Optical Flow**

Optical flow refers to the pattern of **apparent motion** of objects between two consecutive frames, caused by either object movement or camera movement.

### **Key Assumptions in Optical Flow Analysis**
1. **Pixel Intensity Consistency**: The brightness of an object remains the same across frames.
2. **Neighboring Pixels Move Together**: The surrounding pixels of the tracked object move similarly.

### **Optical Flow in OpenCV**
- OpenCV provides methods to track objects between frames.
- The user must **specify points** to track (e.g., detecting a face first, then tracking it).
- The **Lucas-Kanade Method** is used for **sparse optical flow**, meaning it tracks only selected points.
- The **Gunnar Farneback Method** is used for **dense optical flow**, which tracks motion across the entire frame.

### **Tracking Example**
**Note** danish you have to insert picture here for this example.
- A moving ball in consecutive frames could either mean the **ball is moving** or the **camera is shifting**.
- Optical flow algorithms cannot differentiate between object motion and camera movement.
- The function in OpenCV takes:
  - A previous frame $( I_t )$
  - A set of tracking points $( p )$
  - The current frame $( I_{t+1} )$

  to estimate motion.

### **Sparse vs. Dense Optical Flow**
#### **1. Lucas-Kanade Method (Sparse Optical Flow)**
- Tracks only a few selected points.
- Ideal for tracking specific objects, like facial features.

#### **2. Gunnar Farneback Method (Dense Optical Flow)**
- Tracks motion for **all pixels** in an image.
- Highlights moving objects with colors indicating direction.

### **Mathematical Background**
- These methods rely on **linear algebra** and **calculus**.
- The Lucas-Kanade method is based on solving the following equation:

  $$
  I_x u + I_y v + I_t = 0
  $$

  where:
  - $( I_x, I_y )$ are image gradients,
  - $( I_t )$ is the temporal gradient,
  - $( (u, v) )$ represents the optical flow vector.

- The Gunnar Farneback method uses polynomial expansion to estimate flow vectors.

The lecture concludes with implementing both **sparse** and **dense** optical flow in OpenCV.


# Optical Flow Using Lucas-Kanade Method







## Detecting Feature Points
We start by detecting key points in the first frame using **Shi-Tomasi Corner Detection**. This helps us find strong feature points (like corners) that are easier to track across frames. These points are chosen because they have high contrast in multiple directions, making them more reliable for motion tracking

In [ ]:
## Parameters to detect corners
corner_track_params = dict(maxCorners = 10,qualityLevel = 0.3,MinDistance = 7,blockSize = 7)

## Setting Parameters for Optical Flow
We define key parameters that influence the tracking accuracy:
- **Window size**: Determines the local area used for computing optical flow.
- **Pyramid levels**: Enables multi-scale tracking, allowing for better handling of large motions.
- **Termination criteria**: Defines when the iterative optimization process should stop.

These parameters ensure that the **Lucas-Kanade method** tracks points effectively while maintaining performance.

In [ ]:
## Parameters for Lucas-kanade flow motion.
## Smaller window more sensitive to noise and could be missed important information(larger motion between frames).
## By selecting larger, we can catch that larger motion between frames.

lk_params = dict(winSize=(200,200),maxLevel = 2,criteria = (cv2.TERM_CRITERIA_EPS|cv2.TERM_CRITERIA_COUNT,10,0.03))


## Reading Video Frames and Preprocessing
We process the video frame by frame. Each frame is **converted to grayscale** because optical flow operates on intensity values rather than color. This reduces computational complexity and improves accuracy.

## Calculating Optical Flow
Using OpenCV’s `calcOpticalFlowPyrLK()`, we estimate the movement of feature points from the previous frame to the current one. This function works by analyzing pixel intensity variations within the predefined window size and across pyramid levels, solving for the displacement of tracked points.

## Filtering Tracked Points
Some feature points may get lost due to factors like **occlusion, motion blur, or changes in lighting conditions**. We apply filtering techniques to remove invalid points and retain only the successfully tracked ones.

## Visualizing Motion
To visualize the motion, we **draw lines connecting the previous and new positions** of the tracked points. This creates a motion trail effect, allowing us to see the movement of objects in the video. The visualization helps in understanding how objects or the camera itself is moving.

In [ ]:
cap = cv2.VideoCapture(0)

ret, prev_frame = cap.read()
prev_gray = cv2.cvtColor(prev_frame,cv2.COLOR_BGR2GRAY)

## POINTS TO TRACK

prevPts = cv2.goodFeaturesToTrack(prev_gray,mask=None,**corner_track_params)
## Drawing lines on the video
mask = np.zeros_like(prev_frame)

while True:
  ret,frame = cap.read()
  frame_gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

  nextPts,status,err = cv2.calcOpticalFlowPyrLK(prev_gray,frame_gray,prevPts,None,**lk_params)
  good_new = nextPts[ status == 1]
  good_prev = prevPts[ status == 1]

  for i,(new,prev) in enumerate(zip(good_new,good_prev)):
    x_new,y_new = new.ravel()
    x_prev,y_prev = prev.ravel()

    mask = cv2.line(mask,(x_new,y_new),(x_prev,y_prev),(0,255,0),3)
    frame = cv2.circle(frame,(x_new,y_new),8,(0,0,255),-1)

  img = cv2.add(frame,mask)
  cv2.imshow('tracking'.img)

  k = cv2.waitKey(30) & 0xFF
  if k == 27:
    break
  prev_gray = frame_gray.copy()
  prevPts = good_new.reshape(-1,1,2)


cv2.destroyAllWindows()
cap.release()


# **Dense Optical Flow in OpenCV**

This explanation is about **Dense Optical Flow** in OpenCV, which is used to track movement across an entire image rather than just a few points, as in **Lucas-Kanade Optical Flow**.

## **Step-by-Step Explanation**

### **1. Capturing Video Input**
- The program captures a **video stream**, typically from a **USB camera**, using OpenCV.
- The **first frame** of the video is read and converted to **grayscale**, since optical flow works better with **single-channel images**.

### **2. Setting Up HSV Mask**
- The program creates an **HSV (Hue, Saturation, Value) mask** to colorize movement **based on direction**.
- This helps visualize movement, where different directions will be represented in **different colors**.

### **3. Reading Frames & Computing Optical Flow**
- The next frame of the video is captured and converted to **grayscale**.
- OpenCV’s **Farneback Optical Flow** function is used:
  $$
  \text{cv2.calcOpticalFlowFarneback}
  $$
  - Computes dense optical flow across the **whole image**.
  - Uses several parameters (like **pyramid levels**, **window size**, **iterations**, etc.) to optimize flow calculation.

### **4. Converting Flow Data**
- The flow data consists of **X and Y displacement vectors** for each pixel.
- These vectors are converted to **polar coordinates** using:
  $$
  \text{cv2.cartToPolar(dx, dy)}
  $$
  - **Angle** $(\theta)$ determines the **hue (color)** in the HSV mask.
  - **Magnitude** $(r)$ determines the **saturation (intensity of movement)**.

### **5. Applying Color Mapping**
- The **angle** values are mapped to **hue (color)** values:
  $$
  H = \frac{\theta \cdot 180}{\pi \cdot 2}
  $$
- The **magnitude** values are normalized to fit within **0-255**:
  $$
  V = \frac{\text{magnitude} - \text{min}}{\text{max} - \text{min}} \times 255
  $$
- The **HSV mask** is then converted back to **BGR format** for display.

### **6. Displaying the Output**
The result is shown using **cv2.imshow()**, where:

- **Objects moving to the left** → **Blue**.
- **Objects moving to the right** → **Red**.
- **Objects moving up** → **Purple**.
- **Objects moving down** → **Green**.


### **7. Looping & Exiting**
- The process continues in a **loop** until the user **presses the Escape key** (`27`).
- The program then **releases the video capture** and **closes all windows**.

## **Main Takeaways**
- **Dense Optical Flow** analyzes movement for **all pixels**, rather than selected **feature points**.
- **Color mapping** helps **visualize movement directions**.
- OpenCV’s **Farneback method** is used for **efficient optical flow computation**.
- This technique is useful in:
  - **Motion tracking**
  - **Video stabilization**
  - **Object detection in videos**


In [ ]:
cap = cv2.VideoCapture(0)

ret, frame1 = cap.read()
prevImg = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)

hsv_mask = np.zeros_like(frame1)
hsv_mask[:,:,1] = 255



while True:
  ret,frame2 = cap.read()
  nextImg = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

  flow = cv2.calcOpticalFlowFarneback(prevImg,nextImg,None,0.5,3,15,3,5,1.2,0)

  mag,ang = cv2.cartToPolor(flow[:,:,0],flow[:,:,1],angleInDegrees=True)

  hsv_mask[:,:,0] = ang/2
  hsv_mask[:,:,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)

  bgr = cv2.cvtColor(hsv_mask,cv2.COLOR_HSV2BGR)
  cv2.imshow('frame',bgr)

  k = cv2.waitKey(30) & 0xFF
  if k == 27:
    break
  prevImg = nextImg.copy()


cv2.destroyAllWindows()
cap.release()
